## **LOCACIONES PLANTAS CLIENTES**

In [20]:
import gurobipy as gp
from gurobipy import GRB
import pandas
import plotly.graph_objects as go

In [21]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA", index_col=[0, 1])
df_demanda = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES", index_col=0)
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS", index_col=0)

**Conjuntos**

$$ P: \text{Plantas} = \{1, 2,...,30\} $$
$$ C: \text{Clientes} = \{1, 2,...,50\} $$

In [22]:
PLANTAS = df_plantas.index.to_list()
CLIENTES = df_demanda.index.to_list()

**Parámetros**

$$ \text{CostoApertura}_{p}: \text{Costo de abrir la planta } p\in P $$
$$ \text{Capacidad}_{p}: \text{Capacidad máxima de la planta } p\in P $$
$$ \text{Demanda}_{c}: \text{Demanda del cliente } c\in C $$
$$ \text{CostoAsignacion}_{p,c}: \text{Costo de asignar la planta } p\in P \text{ al cliente } c\in C $$

In [23]:
model = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-28


**Variables**

$$ X_{p} = \left\{ \begin{array}{cl}
1: \text{Si abro la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$
$$ Y_{pc} = \left\{ \begin{array}{cl}
1: \text{Si asigno el cliente } c \in C \text{ a la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$

In [24]:
x = model.addVars(PLANTAS, vtype=GRB.BINARY, name="x")
y = model.addVars([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], vtype=GRB.BINARY, name="y")

**Función Objetivo**

$$ \text{minimizar } \text{FO} = \sum_{p\in P} \text{CAP}_{p} \times X_{p} + \sum_{p\in P} \sum_{c\in C} \text{CCP}_{pc} \times Y_{pc} $$

In [25]:
costo_abrir_plantas = gp.quicksum(x[planta] * df_plantas.loc[planta, "Costo de abrir"] for planta in PLANTAS)
costo_cliente_planta = gp.quicksum(y[(planta, cliente)] * df_costo_cliente_planta.loc[(planta, cliente), "Costo de asignar un cliente a un planta"] for planta in PLANTAS for cliente in CLIENTES)
model.setObjective(costo_abrir_plantas + costo_cliente_planta, GRB.MINIMIZE)

**Restricciones**

$$ \forall p\in P \, \forall c\in C: Y_{pc} \le X_{p} $$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \le \text{CAPAC}_{p} $$

$$ \forall c\in C: \sum_{p\in P} Y_{pc} = 1 $$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \ge 3 X_{p}$$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \ge 0.9 * \text{CAPAC}_{p} X_{p} $$

In [27]:
for planta in PLANTAS:
    for cliente in CLIENTES:
        model.addConstr(y[(planta, cliente)] <= x[planta], name=f"ct1_{planta}_{cliente}")

for planta in PLANTAS:
    model.addConstr(gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) <= df_plantas.loc[planta, "Capacidad máxima"], name=f"ct2_{planta}")

for cliente in CLIENTES:
    model.addConstr(gp.quicksum(y[(planta, cliente)] for planta in PLANTAS) == 1, name=f"ct3_{cliente}")

for planta in PLANTAS:
    model.addConstr(gp.quicksum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], name=f"ct4_{planta}")

for planta in PLANTAS:
    model.addConstr(gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) >= 0.9 * df_plantas.loc[planta, "Capacidad máxima"] * x[planta], name=f"ct5_{planta}")



In [28]:
model.setParam("TimeLimit", 20)
model.optimize()

Set parameter TimeLimit to value 20
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1110 rows, 1020 columns and 6040 nonzeros
Model fingerprint: 0xbd23f1ae
Variable types: 0 continuous, 1020 integer (1020 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 220 rows and 74 columns
Presolve time: 0.12s
Presolved: 890 rows, 946 columns, 6411 nonzeros
Variable types: 0 continuous, 946 integer (946 binary)

Root relaxation: objective 6.319329e+03, 1033 iterations, 0.07 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [55]:
solution = model.ObjVal
print(f"Optimal objective value: {solution}")

df_resultados = pandas.DataFrame()
PLANTAS_ABIERTAS = {}
for planta in PLANTAS:
    if x[planta].X > 0.99:
        print(f"=== Planta {planta} ====")
        print(f"Capacidad Planta: {df_plantas.loc[planta, 'Capacidad máxima']}")
        print(f"Cantidad Enviada: {sum(round(y[(planta, cliente)].X, 0) * df_demanda.loc[cliente, 'Demanda'] for cliente in CLIENTES)}")
        CLIENTES_ATENDIDOS = []
        for cliente in CLIENTES:
            if y[(planta, cliente)].X > 0.99:
                print(cliente, end="  ")
                CLIENTES_ATENDIDOS.append(cliente)
                df_resultados.loc[cliente, planta] = 1
        print("\n")
        PLANTAS_ABIERTAS[planta] = {
            "CAPACIDAD_PLANTA": df_plantas.loc[planta, "Capacidad máxima"],
            "CANTIDAD_ENVIADA": sum(round(y[(planta, cliente)].X, 0) * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES),
            "CLIENTES_ATENDIDOS": CLIENTES_ATENDIDOS,
        }

Optimal objective value: 39747.0
=== Planta 3 ====
Capacidad Planta: 32
Cantidad Enviada: 32.0
7  20  49  

=== Planta 4 ====
Capacidad Planta: 52
Cantidad Enviada: 52.0
9  10  29  

=== Planta 6 ====
Capacidad Planta: 95
Cantidad Enviada: 95.0
3  47  50  

=== Planta 7 ====
Capacidad Planta: 29
Cantidad Enviada: 29.0
11  35  41  

=== Planta 9 ====
Capacidad Planta: 80
Cantidad Enviada: 80.0
17  19  32  36  

=== Planta 10 ====
Capacidad Planta: 97
Cantidad Enviada: 97.0
4  14  43  

=== Planta 11 ====
Capacidad Planta: 23
Cantidad Enviada: 23.0
8  22  37  

=== Planta 12 ====
Capacidad Planta: 85
Cantidad Enviada: 85.0
1  6  42  

=== Planta 13 ====
Capacidad Planta: 72
Cantidad Enviada: 72.0
13  21  28  

=== Planta 14 ====
Capacidad Planta: 62
Cantidad Enviada: 62.0
15  25  33  40  

=== Planta 15 ====
Capacidad Planta: 48
Cantidad Enviada: 46.0
5  26  48  

=== Planta 16 ====
Capacidad Planta: 100
Cantidad Enviada: 100.0
12  30  45  46  

=== Planta 17 ====
Capacidad Planta: 77
Ca

In [56]:
df_resultados = df_resultados.fillna(0)
df_resultados = df_resultados.sort_index()
df_resultados.head()

,3,4,6,7,9,10,11,12,13,14,15,16,17,18,19
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [80]:
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=df_resultados.values,
        x=df_resultados.columns.map(
            lambda i: f"P{i:02d} - {PLANTAS_ABIERTAS[i]['CAPACIDAD_PLANTA']}<br />CA: {len(PLANTAS_ABIERTAS[i]['CLIENTES_ATENDIDOS'])}<br />{PLANTAS_ABIERTAS[i]['CAPACIDAD_PLANTA'] / PLANTAS_ABIERTAS[i]['CAPACIDAD_PLANTA']:.0%}"
        ),
        y=df_resultados.index.map(lambda i: f"C{i:02d}"),
        colorscale="greens",
        showscale=False,
    )
)

# Añadiendo anotaciones donde el valor es 1
annotations = []
for i in range(df_resultados.shape[0]):
    for j in range(df_resultados.shape[1]):
        if df_resultados.iat[i, j] == 1:
            c = df_resultados.index[i]
            annotations.append(
                go.layout.Annotation(
                    x=j,
                    y=i,
                    text=f"C{c:02d} - D: {df_demanda.loc[c, 'Demanda']}",
                    showarrow=False,
                    font=dict(color="white", size=10),
                    align="center",
                )
            )

# Actualizar el diseño de la figura
fig.update_layout(
    title="ASIGNACIÓN",
    height=1000,
    width=1200,
    template="ggplot2",
    annotations=annotations,
    xaxis=dict(title=None, side="top"),
    yaxis=dict(title="Clientes", autorange="reversed"),
    margin=dict(t=150),
)

fig.show()